In [22]:
import os
import requests
import torch


from dotenv import load_dotenv
from huggingface_hub import login

from transformers import AutoTokenizer, \
            AutoModelForCausalLM, \
            TextStreamer, \
            Wav2Vec2Processor, \
            Wav2Vec2ForCTC

from IPython.display import Markdown, display, update_display, Audio

In [13]:
!pip install librosa

  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 22.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   ------------- -------------------------- 9.2/28.1 MB 40.9 MB/s eta 0:00:01
   --------------------------- ------------ 19.7/28.1 MB 46.0 MB/s eta 0:00:01
   ---------------------------------------  28.0/28.1 MB 49.4 MB/s eta 0:00:01
   ---------------------------------------- 28.1/28.1 MB 42.4 MB/s eta 0:00:00


In [17]:
import librosa as lb

In [18]:
load_dotenv()

In [19]:
login(os.getenv('HF_TOKEN'), add_to_git_credential=True)

Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\ma2\.cache\huggingface\token
Login successful


In [20]:
LLAMA =  'meta-llama/Meta-Llama-3.1-8B-Instruct'
FB_WAV_MODEL = 'facebook/wav2vec2-base-960h'

In [21]:
# Initialize the processor
processor = Wav2Vec2Processor.from_pretrained(FB_WAV_MODEL)

# Initialize the model
model = Wav2Vec2ForCTC.from_pretrained(FB_WAV_MODEL)

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

c:\Users\ma2\.conda\envs\generative-llm-projects\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ma2\.cache\huggingface\hub\models--facebook--wav2vec2-base-960h. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

c:\Users\ma2\.conda\envs\generative-llm-projects\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
audio_file = './audio/story.mp3'

In [28]:
Audio(audio_file, autoplay=True)

In [29]:
# Loading the clean audio file
wf_clean, sr_clean = lb.load(audio_file, sr = 16000)

# Tokenize the waveform generated from the clean audio using librosa
input_clean = processor(wf_clean, return_tensors='pt').input_values


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [31]:
# retrieve the logits from the model
logts_clean = model(input_clean).logits
    

In [32]:
# We will take the argmax value from the logits retrieved from the model
pred_clean = torch.argmax(logts_clean, dim=-1)
transcription_clean = processor.batch_decode(pred_clean)

In [33]:
print(transcription_clean)

["FUN IN OSAKA THE OSAKA DEMONSTRATION WAS GREAT FUN I PARTICULARLY ENJOYED MEETING SOME OF OUR EFFORTLESS ENGLISH MEMBERS IN JAPAN MY ONLY REGRET IS THAT I DIDN'T HAVE TIME TO TALK TO EVERYONE LONGER NEXT TIME WILL SCEGIA MORE TIME TO JUST SIT AND CHAP SINCE THE OSAKA DEMONSTRATION WENT WELL WI'LL BE DOING MORE WORKSHOPS AND SEMINARS IN THE FUTURE MY PLAN IS TO DEVELOP TWO KINDS OF SEMINARS ONE FOR ENGLISH TEACHERS IN ONE FOR ENGLISH LEARNERS THERE'S A HUGE NEED FOR TEACHER TRAINING AS MOST ENGLISH TEACHERS EITHER HAVE NO TRAINING AT ALL OR THEY ARE TRAINED IN GRAMMAR ANALYSIS PRACTISE METHODS IN OTHER WORDS THEY ARE ONLY TRAINED TO USE TEXT BOOKS LIKEWISE I BELIEVE THERE IS A NEED FOR STUDENT TRAINING MOST STUDENTS DON'T KNOW ABOUT THE RESEARCH THEY DON'T KNOW THERE ARE OTHER WAYS TO LEARN ENGLISH THEY ONLY KNOW WHAT THEY HAVE EXPERIENCE IN SCHOOL USUALLY BORING GRAMMAR BAYSE TEXT BOOK TEACHING MI GHAEL FOR STUDENT SEMINARS IS TO TEACH STUDENTS A NEW WAY TO LEARN INDEPENDENTLY THE OS

In [34]:
system_message = "You are an assistant that produces minutes of meetings or stories from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a story. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription_clean}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [37]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA,trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt').to('cpu')

model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map='auto')
output = model.generate(inputs, max_new_tokens=2000)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [39]:
response = tokenizer.decode(output[0])

In [40]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings or stories from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a story. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
["FUN IN OSAKA THE OSAKA DEMONSTRATION WAS GREAT FUN I PARTICULARLY ENJOYED MEETING SOME OF OUR EFFORTLESS ENGLISH MEMBERS IN JAPAN MY ONLY REGRET IS THAT I DIDN'T HAVE TIME TO TALK TO EVERYONE LONGER NEXT TIME WILL SCEGIA MORE TIME TO JUST SIT AND CHAP SINCE THE OSAKA DEMONSTRATION WENT WELL WI'LL BE DOING MORE WORKSHOPS AND SEMINARS IN THE FUTURE MY PLAN IS TO DEVELOP TWO KINDS OF SEMINARS ONE FOR ENGLISH TEACHERS IN ONE FOR ENGLISH LEARNERS THERE'S A HUGE NEED FOR TEACHER TRAINING AS MOST ENGLISH TEACHERS EITHER HAVE NO TRAINING AT ALL OR THEY ARE TRAINED IN GRAMMAR ANALYSIS PRACTISE METHODS IN OTHER WORDS THEY ARE ONLY TRAINED TO USE TEXT BOOKS LIKEWISE I BELIEVE THERE IS A NEED FOR STUDENT TRAINING MOST STUDENTS DON'T KNOW ABOUT THE RESEARCH THEY DON'T KNOW THERE ARE OTHER WAYS TO LEARN ENGLISH THEY ONLY KNOW WHAT THEY HAVE EXPERIENCE IN SCHOOL USUALLY BORING GRAMMAR BAYSE TEXT BOOK TEACHING MI GHAEL FOR STUDENT SEMINARS IS TO TEACH STUDENTS A NEW WAY TO LEARN INDEPENDENTLY THE OSAKA SEMINAR WAS MY FIRST ONE IT WENT WELL BUT I FELT IT WAS A BIT RUFGH AND NEEDS IMPROVEMENT WHEN I GET BACK TO SAN FRANCISCO I WILL START DOING REGULAR SEMINARS IN THE CITY IN THIS WAY I CAN PRACTISE AND IMPROVE BOTH VERSIONS OF THE SEMINAR ONCE I'M SATISFIED WITH THE QUALITY WE WILL GO ON TOUR AND DO SEMINARS IN DIFFERENT CITIES AROUND THE WORLD"]<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of Meeting**
======================

**Summary**
-----------

* Date: Not specified
* Location: Osaka, Japan
* Attendees: Not specified
* Summary: The meeting was a discussion on the success of the Osaka demonstration and future plans for workshops and seminars.

**Discussion Points**
--------------------

* The Osaka demonstration was a great success and will lead to more workshops and seminars in the future.
* Two types of seminars will be developed: one for English teachers and one for English learners.
* There is a huge need for teacher training as most English teachers have no training or are only trained in grammar analysis and textbook-based methods.
* There is also a need for student training as most students are not aware of alternative ways to learn English.
* The student seminars will focus on teaching students a new way to learn independently.
* The Osaka seminar was the first one and needs improvement, but will be practiced and improved upon in San Francisco.

**Takeaways**
-------------

* The Osaka demonstration was a success and will lead to more workshops and seminars.
* Two types of seminars will be developed to address the needs of English teachers and learners.
* There is a need for teacher and student training to improve English learning.

**Action Items**
---------------

* **Owner: Speaker**
  * Develop two types of seminars: one for English teachers and one for English learners.
  * Improve the Osaka seminar and start doing regular seminars in San Francisco.
  * Once satisfied with the quality, go on tour and do seminars in different cities around the world.
  * Start developing the student seminars to teach students a new way to learn independently.
* **Owner: Team**
  * Provide support and resources for the development of the seminars.
  * Help improve the Osaka seminar and provide feedback for future improvements.<|eot_id|>